# Evaluating Regression Lines Lab

### Introduction

In the previous lesson, we learned to evaluate how well a regression line estimated our actual data.  In this lab, we will turn these formulas into code.

### Determining Quality

In the file, `movie_data.py` you will find movie data written as a python list of dictionaries, with each dictionary representing a movie.  The movies are derived from the first 30 entries from the 538 dataset [provided here](https://raw.githubusercontent.com/fivethirtyeight/data/master/bechdel/movies.csv).

In [4]:
from movie_data import movies 
len(movies)

30

In [5]:
movies[0]

{'budget': 13000000, 'domgross': 25682380.0, 'title': '21 &amp; Over'}

Note that like in previous lessons, we are still referencing a budget as our `x_values` or explanatory value and our revenue as our ouput.  Here revenue is represented as the key `domgross`.  First, convert the explanatory budget values to `x_values`, and convert the domgross values to `y_values`.

In [6]:
x_values = list(map(lambda movie: movie['budget'], movies))
y_values = list(map(lambda movie: movie['domgross'], movies))

In [7]:
x_values[0] # 13000000

13000000

In [8]:
y_values[0]

25682380.0

Assign a variable called `titles` equal to the titles of the movies.

In [13]:
titles = list(map(lambda movie: movie['title'], movies))

In [17]:
titles[0]

'21 &amp; Over'

For the dataset, let's use the following regression formula: $\overline{y} = \overline{m} x + \overline{b}$, with $\overline{m} = 1.7$, and $\overline{y} = 100,000$.  Write a function called `regression_formula` that calculates our $\overline{y}$ for any provided value of $x$. 

In [9]:
def regression_formula(x):
    return 100000 + 1.7*x

In [10]:
regression_formula(100000) # 270000.0
regression_formula(250000) # 525000.0

525000.0

Let's plot the data as well as the regression line to get a sense of what we are looking at.

In [16]:
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

from graph import trace_values, m_b_trace, plot
movies_trace = trace_values(x_values, y_values, text=titles, name='movie data')
regression_trace = m_b_trace(1.7, 100000, x_values, name='estimated revenue')
plot([movies_trace, regression_trace])

### Calculating errors of a regression Line

Now that we have our regression formula, we can move towards calculating the error.  Let's start by writing a function named `y_actual`, that given a value, $x$, a list of `x_values` and a list of `y_values` to represent our dataset, returns the value of $y$ at $x$.

In [23]:
def y_actual(x, x_values, y_values):
    combined_values = list(zip(x_values, y_values))
    point_at_x = list(filter(lambda point: point[0] == x,combined_values))[0]
    return point_at_x[1]

In [24]:
y_actual(13000000, x_values, y_values) # 25682380.0

25682380.0

Now write a function called `error`, that given a list of `x_values`, and a list of `y_values`, the values `m` and `b` of a regression line, and a value of `x`, returns the error at that x value.  Remember ${\varepsilon_i} =  y_i - \overline{y}_i$.  

In [25]:
def error(x_values, y_values, m, b, x):
    expected = (m*x + b)
    return (y_actual(x, x_values, y_values) - expected)

In [26]:
error(x_values, y_values, 1.7, 100000, 13000000) # 3482380.0

3482380.0

Now that we have a formula to calculate our errors, write a function that returns a trace of an error at a given point.  The function is called `error_line` and it takes our dataset of `x_values` as the first argument, and `y_values` as the second argument.  It also takes in values of $m$ and $b$ as the next two arguments, to represent the regression line it is calculating errors from.  Finally, it's last argument is the value $x$ it is drawing an error for.

The return value is a dictionary that represents a trace.  The x values of the trace are two x values, and the y values of the trace are the actual value and the expected value.  The mode of the trace should equal `line`.

In [107]:
def error_line_trace(x_values, y_values, m, b, x):
    y_hat = m*x + b
    y = y_actual(x, x_values, y_values)
    name = 'error at ' + str(x)
    return {'x': [x, x], 'y': [y, y_hat], 'mode': 'line', 'marker': {'color': 'red'}, 'name': name}

In [108]:
error_at_120m = error_line_trace(x_values, y_values, 1.7, 10000, 120000000)

# {'marker': {'color': 'red'},
#  'mode': 'line',
#  'name': 'error at 120000000',
#  'x': [120000000, 120000000],
#  'y': [93050117.0, 204010000.0]}
error_at_120m

{'marker': {'color': 'red'},
 'mode': 'line',
 'name': 'error at 120000000',
 'x': [120000000, 120000000],
 'y': [93050117.0, 204010000.0]}

We just ran the our function to draw a trace of the error for the movie Elysium.  Let's see how it looks.

In [42]:
movies[19]

{'budget': 120000000, 'domgross': 93050117.0, 'title': 'Elysium'}

In [31]:
from linear_equations import expected_value_for_line
y_actual(13000000, x_values, y_values) # 25682380.0
expected_value_for_line(1.7, 10000, 13000000) # 22110000.0

22110000.0

In [45]:
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

from graph import trace_values, m_b_trace, plot
movies_trace = trace_values(x_values, y_values, text=titles, name='movie data')
regression_trace = m_b_trace(1.7, 100000, x_values, name='estimated revenue')
plot([movies_trace, regression_trace, error_at_120m])

From there, we can write a function called `error_lines`, that takes in a list of `x_values` as an argument, `y_values` as an argument, and returns a list of traces for every x_value provided.

In [113]:
def error_line_traces(x_values, y_values, m, b):
    return list(map(lambda x_value: error_line_trace(x_values, y_values, m, b, x_value), x_values))

In [112]:
errors_for_regression = error_line_traces(x_values[0:5], y_values[0:5], 1.7, 100000)

In [109]:
len(errors_for_regression) # 5
errors_for_regression[-1]

# {'marker': {'color': 'red'},
#  'mode': 'line',
#  'name': 'error at 40000000',
#  'x': [40000000, 40000000],
#  'y': [95020213.0, 68100000.0]}

{'marker': {'color': 'red'},
 'mode': 'line',
 'name': 'error at 40000000',
 'x': [40000000, 40000000],
 'y': [95020213.0, 68100000.0]}

In [59]:
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

from graph import trace_values, m_b_trace, plot
movies_trace = trace_values(x_values, y_values, text=titles, name='movie data')
regression_trace = m_b_trace(1.7, 100000, x_values, name='estimated revenue')
plot([movies_trace, regression_trace, *errors_for_regression])

### Calculating RSS

Now write a function called `squared_error`, that given a value of x, returns the squared error at that x value.

${\varepsilon_i}^2 =  (y_i - \overline{y}_i)^2$

In [67]:
def squared_error(x_values, y_values, m, b, x):
    return error(x_values, y_values, m, b, x)**2

In [68]:
squared_error(x_values, y_values, 1.7, 100000, x_values[0]) # 12126970464400.0

12126970464400.0

Next, write a function called `residual_sum_squares` that provided a list of points returns the total squared error for the movies in our dataset.

In [69]:
def squared_errors(x_values, y_values, m, b):
    return list(map(lambda x: squared_error(x_values, y_values, m, b, x), x_values))

In [70]:
squared_errors(x_values, y_values, 1.7, 100000)

[12126970464400.0,
 4135150451673360.5,
 361267379491225.0,
 794537411251600.0,
 724697867965369.0,
 1.1849947361812563e+17,
 7947865497243204.0,
 26791793814241.0,
 12126970464400.0,
 2.578526293187741e+16,
 724697867965369.0,
 28038359355876.0,
 4309641785171044.0,
 7000432263889.0,
 183234585197889.0,
 250695953891161.0,
 170556704389696.0,
 5.700890907419822e+16,
 225831887511616.0,
 1.2332076514313688e+16,
 1.5715833880370482e+16,
 3916421362617124.0,
 724697867965369.0,
 8814749428288609.0,
 637050330900736.0,
 1116906426022500.0,
 1.9030233952612996e+16,
 1.33580290597636e+16,
 3147108684215409.0,
 250695953891161.0]

In [71]:
def residual_sum_squares(x_values, y_values, m, b):
    return sum(squared_errors(x_values, y_values, m, b))

residual_sum_squares(x_values, y_values, 1.7, 100000)

3.0025171100327725e+17

Now we'll provide a couple functions for you. a function called `trace_rss`, that build a bar chart displaying the value of the RSS.  The return value is a dictionary with keys of `x` and `y`, both which point to lists.  The $x$ key should point to a list with one element, the string 'RSS'.  The $y$ list should be a list which points to the value of the RSS for the line.

In [110]:
import plotly.graph_objs as go

def trace_rss(x_values, y_values, m, b):
    rss_calc = residual_sum_squares(x_values, y_values, m, b)
    return dict(
        x=['RSS'],
        y=[rss_calc],
        type='bar'
    )

trace_rss(x_values, y_values, 1.7, 100000)

{'type': 'bar', 'x': ['RSS'], 'y': [3.0025171100327725e+17]}

Once this is built, we can turn build a subplot showing the regression line, as well as the related RSS for the regression line.

In [104]:
import plotly
from plotly import tools
import plotly.graph_objs as go

def plot_regression_and_rss(scatter_trace, regression_trace, rss_calc_trace):
    fig = tools.make_subplots(rows=1, cols=2)
    fig.append_trace(scatter_trace, 1, 1)
    fig.append_trace(regression_trace, 1, 1)
    fig.append_trace(rss_calc_trace, 1, 2)
    plotly.offline.iplot(fig)

In [114]:
m = 1.7
b = 100000


scatter_trace = trace_values(x_values, y_values, text=titles, name='movie data')
regression_trace = m_b_trace(m, b, x_values, name='estimated revenue')
rss_calc_trace = trace_rss(x_values, y_values, m, b)

plot_regression_and_rss(scatter_trace, regression_trace, rss_calc_trace)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]

